In [ ]:
import os
import pandas as pd
import google_auth_oauthlib.flow
import googleapiclient.discovery
import youtube_dl
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from credentials import client_id, client_secret, user_id, username

In [ ]:
# Spotify creds
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret

redirect_uri = "http://localhost:7777/callback"  #Spotify will open a grant data permission page here
os.environ['SPOTIPY_REDIRECT_URI'] = redirect_uri

# Youtube 
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Set this to 1 only when running locally
api_service_name = "youtube"
api_version = "v3"
CLIENT_SECRETS = "client_secret.json"  #youtube credentials file

In [ ]:
# Initialize youtube data api client
def init_youtube_client(client_secrets_file):

    # Get credentials and create an API client
    scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
    credentials = flow.run_console()  # This will ask to authenticate and paste access token

    # from the Youtube DATA API
    youtube_client = googleapiclient.discovery.build(api_service_name, 
                                                     api_version, 
                                                     credentials=credentials)
    
    return youtube_client


# Initialize spotify client credentials to get non-user data
def init_spotify_client():
    spotify_client = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())    
    return spotify_client


# Get liked videos from youtube and their corresponding spotify track uri
def get_liked_vids_uris():  
    
    youtube_client = init_youtube_client(CLIENT_SECRETS)
    spotify_client = init_spotify_client()
    
    likes = youtube_client.videos()
    request = likes.list(part="snippet,contentDetails", myRating="like")
    music_df = pd.DataFrame(columns = ['video_title', 'song_name', 'artist', 'uri'])
    
    while request is not None:
        liked_vids = request.execute()
        for item in liked_vids["items"]:
            video_title = item["snippet"]["title"]
            youtube_url = f'https://www.youtube.com/watch?v={item["id"]}'

            # use youtube_dl to collect the song name & artist name
            try:
                video = youtube_dl.YoutubeDL({}).extract_info(youtube_url, download=False)
            except Exception as e:
                print(f"Error getting song info, skipping {video_title}")
                continue
            else:
                song_name = video["track"]
                artist = video["artist"]
                if((song_name is not None) & (artist is not None)):
                    print(" -- ".join([video_title, song_name, artist]))

                    # Get spotify URI of track
                    result = spotify_client.search(q=f'track:{song_name} artist:{artist}', type='track')   
                    
                    # Pick the top match in case there are multiple matches
                    for song in result['tracks']['items']:
                        
                        uri = song['uri'].split(":")[-1]
                        row = {'video_title':video_title, 
                                'song_name':song['name'], 
                                'artist':song['artists'][0]['name'],
                                'uri':uri}
                        print(row)
                        
                        music_df = music_df.append(row, ignore_index=True)
                        break
                        
        request = likes.list_next(request, liked_vids)
    return music_df


# Create a new spotify playlist to store the songs
def create_playlist(playlist_name):
    
    ''' Create a new spotify playlist to store the songs '''
    
    token = util.prompt_for_user_token(username=username, 
                                       scope="playlist-modify-public")

    if token:
        sp = spotipy.Spotify(auth=token)
        sp.user_playlist_create(user_id, playlist_name)  # Create new playlist
        print("Playlist created.")
    else:
        print("Can't get token for", username)
        return None

    
def get_playlist_uri(playlist_name=None):
    
    ''' Display all the playlists for the user and get the uri of given playlist '''
    
    token = util.prompt_for_user_token(username=username, 
                                       scope='user-library-read')

    # Show all playlist and get the uri
    sp = spotipy.Spotify(auth=token)
    playlists = sp.user_playlists(user_id)
    uri = None
    for item in playlists['items']:
        print(item['name'], "--", item['uri'])
        if(item['name'] == playlist_name):
            uri = item['uri'].split(":")[-1]
    
    if uri is not None:
        return uri
    else:
        return None
    
    
def add_tracks(playlist_uri, tracks):
    
    ''' Add tracks to playlist '''
    
    token = util.prompt_for_user_token(username, scope='playlist-modify-public')
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        results = sp.user_playlist_add_tracks(username, playlist_uri, tracks)
        print(results)   
    else:
        print("Cant get token for", username)


**Get liked videos from youtube**

In [ ]:
music_df = get_liked_vids_uris()

In [ ]:
music_df.tail()

** Add tracks to a new playlist **

In [ ]:
# Create a new playlist to add the tracks
playlist_name = 'Youtube Likes'
playlist_uri = create_playlist(playlist_name)

In [ ]:
playlist_uri = get_playlist_uri(playlist_name)

In [ ]:
# Add tracks to playlist
add_tracks(playlist_uri, list(music_df.tracks))